## **Image Classification Improvement: Text-to-Image AI for Synthetic Dataset Approach**





Steps:

- To check:
  - Edit ->  Notebook setings -> Hardware accelerator: GPU

- To do:
  - Modify the input with the desired animal

  - Select the style

  - Run all the cells

  - Wait for the results!

## **Inputs**

In [ ]:
input = "cat"
# This Models is tuned to produce images of mammals. Hence, if the input is not an mammal, it is possible get bad results.

number_of_images = 200

steps = 50

In [ ]:
### Style 1 ###
prompt = "a hyper realistic photo of a " + input + ", 8k, unreal engine render, vray, vivid, nice texture, detailed, hyper realistic, detailed, high resolution, hdr 4k, proportioned, professional image, realistic eyes, coherent colors, national geographic"

my_negative_prompt = "unrealistic, disfigured, deformed, blurry, undetailed, weird colors, weird eyes, jpg artefacts, missing extremities, additional extremities, weird extremities, poorly drawn, logos, weird fur, unrealistic fur, disfigured mouth, additonal ears" #

###################################

In [ ]:
### Style 2 ###
prompt = "An ultra-realistic image of a " +input+ " that is so lifelike, it could be mistaken for a photograph, the image captures the unique features and personality of the "+input+", realistic fur texture,realistic expression, natural body language, detailed, hyper realistic, detailed, high resolution, hdr 4k, proportioned, professional image, realistic eyes, coherent colors, national geographic"

my_negative_prompt = "clearly artificial, lacks any sense of realism, computer-generated,  jpg artefacts, poorly drawn, logos, unrealistic fur, aditional extrimities, fake eyes" #

## **Loading libraries**


In [ ]:
!nvidia-smi

Tue Jun 20 15:21:56 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#Dowload necessaty files
!pip install --upgrade git+https://github.com/huggingface/diffusers.git@main
!pip install transformers
!pip install accelerate
!pip install scipy
!pip install triton
!pip install ftfy
!pip install gradio -q

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/diffusers.git (to revision main) to /tmp/pip-req-build-8pfvesy3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-8pfvesy3
  Resolved https://github.com/huggingface/diffusers.git to commit 3ebbaf7c96801271f9e6c21400033b6aa5ffcf29
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 9.5 MB/s eta 0:00:00
  Created wheel for diffusers: filename=diffusers-0.18.0.dev0-py3-none-any.whl size=1121816 sha256=91a8f46e75a90f3fb14f869c68a414b91b8e48e55504fe20996f775a0c866321
  Stored in directory: /tmp/pip-ephem-wheel-cache-pf8xaiz7/wheels/2f/e1/9c/c4e22972ee698b020d29ed7cc914d898247d01e302f25df1a4
Successfully built diffusers
Looking in indexes: https:

In [ ]:
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline, StableDiffusionUpscalePipeline, DiffusionPipeline, DPMSolverMultistepScheduler
import gradio as gr
import torch
from PIL import Image
import random
import os

In [ ]:
SAVE_PATH = os.getcwd()


In [ ]:
model_id = 'stabilityai/stable-diffusion-2-1'
# model_id = "runwayml/stable-diffusion-v1-5" # sometimes it probides better resutls
# https://huggingface.co/stabilityai/stable-diffusion-2-1

In [ ]:


num_images_per_prompt = number_of_images

In [ ]:
height = 768
width = 768

device_type = 'cuda' # 'cuda' for GPU, 'cpu' for CPU compute
low_vram = True




def uniquefile(path):
    filename, extension = os.path.splitext(path)
    file_c = 1

    while os.path.exists(path):
        path = filename + ' (' + str(file_c) + ')' + extension
        file_c +=1

    return path

def render_prompt():
    shorted_prompt = (prompt[:50] + '...') if len(prompt) > 25 else prompt
    shorted_prompt = shorted_prompt.replace(' ', '_')

    generation_path = os.path.join(SAVE_PATH, shorted_prompt.removesuffix('...'))

    if not os.path.exists(SAVE_PATH):
        os.mkdir(SAVE_PATH)

    if not os.path.exists(generation_path):
        os.mkdir(generation_path)

    # Call Pipeline:

    scheduler = DPMSolverMultistepScheduler.from_pretrained(model_id, subfolder="scheduler")


    pipe = StableDiffusionPipeline.from_pretrained(
      model_id,
      revision="fp16" if torch.cuda.is_available() else "fp32",
      torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
      scheduler=scheduler
    ).to("cuda")
    pipe.enable_attention_slicing()
    seed = random.randint(0, 2147483647)
    generator = torch.Generator('cuda').manual_seed(seed)


    prompt_stylized = f"{prompt}"
    print(f"Full Prompt:\n{prompt_stylized} \n")
    print(f"Characters in prompt: :{len(prompt)}, limit: 190")

    for i in range(num_images_per_prompt):
          image = pipe(prompt_stylized,num_inference_steps=steps ,negative_prompt= my_negative_prompt,guidance_scale = 7.5, generator= generator).images[0]
          #image = pipe("a hyper realistic photo of a dog", negative_prompt= my_negative_prompt,guidance_scale = 7.5).images[0]


          image_path = uniquefile(os.path.join(SAVE_PATH,generation_path, 'mammal' + " - " +shorted_prompt) + '.png')
          print(image_path)

          image.save(image_path)
    print("\n done!")





## **Render**


In [ ]:
render_prompt()

text_encoder/model.safetensors not found


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

Full Prompt:
An ultra-realistic image of a cat that is so lifelike, it could be mistaken for a photograph, the image captures the unique features and personality of the cat, realistic fur texture,realistic expression, natural body language, detailed, hyper realistic, detailed, high resolution, hdr 4k, proportioned, professional image, realistic eyes, coherent colors, national geographic 

Characters in prompt: :376, limit: 190


  0%|          | 0/50 [00:00<?, ?it/s]

/content/An_ultra-realistic_image_of_a_cat_that_is_so_lifel/mammal - An_ultra-realistic_image_of_a_cat_that_is_so_lifel....png


...

  0%|          | 0/50 [00:00<?, ?it/s]

/content/An_ultra-realistic_image_of_a_cat_that_is_so_lifel/mammal - An_ultra-realistic_image_of_a_cat_that_is_so_lifel... (103).png


  0%|          | 0/50 [00:00<?, ?it/s]

/content/An_ultra-realistic_image_of_a_cat_that_is_so_lifel/mammal - An_ultra-realistic_image_of_a_cat_that_is_so_lifel... (104).png


  0%|          | 0/50 [00:00<?, ?it/s]